In [3]:
from dotenv import load_dotenv

_ = load_dotenv()

In [8]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [14]:
!pip install langgraph-checkpoint-sqlite


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langgraph-checkpoint-sqlite]


In [ ]:
tool = TavilySearchResults(max_results=2)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 28.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [langchain_community]ngchain_community]


In [29]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [ ]:
import requests
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from typing import List, Optional
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
import json 
class UserProfile(BaseModel):
    age: Optional[int]
    gender: Optional[str]
    occupation: Optional[str]

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    user_profile: UserProfile

ONBOARDING_USER_CREATE_PROMPT = """You are the Big Boss bot: a helpful assistant that recommends movies to users.
 In the first step, you will ask user information to create a user profile. The information you need to ask is:
 - age
 - gender
 - occupation

Currently the user profile that you have is:
{user_profile}

Ask the user for the information that you don't have. 
"""


USER_CREATE_STATE_PROMPT = """You are the Big Boss bot: a helpful assistant that recommends movies to users.
You already have a user profile with the following information:
{user_profile}

Now you will thanks the user for the information and ask for the next information.
"""

EXTRACTION_USER_CREATE_PROMPT = """You are an extractor of user profile from a text.
 The text is a conversation between a user and a bot.
 The user profile is a dictionary with the following keys:
 - age
 - gender
 - occupation
You will receive a conversation message and you will extract the user profile from the message. Extract the user profile to a JSON object. 
Example:
```
User: Hello, my name is John Doe. I am 30 years old. I am a software engineer.
Bot: Hello, John. Nice to meet you.
```
The user profile is:
```json
{
    "name": "John Doe",
    "age": 30,
    "gender": "male",
    "occupation": "software engineer"
}
"""


    

In [87]:
class Agent:
    def __init__(self, checkpointer, system=""):
        graph = StateGraph(AgentState)
        graph.add_node("user_profile_prompt", self.user_profile_prompt)
        graph.add_node("user_profile_extraction", self.user_profile_extraction)
        graph.add_edge("user_profile_prompt", "user_profile_extraction")
        graph.add_conditional_edges("user_profile_extraction", self.exists_user_profile, {True: END, False: "user_profile_prompt"})
        graph.set_entry_point("user_profile_prompt")

        self.graph = graph.compile(checkpointer=checkpointer)
        self.model = ChatOpenAI(model="gpt-5-nano", temperature=0)

    def exists_user_profile(self, state: AgentState) -> bool:
        user_profile = state.get("user_profile")
        if user_profile is None:
            return False
        return user_profile.age is not None and user_profile.gender is not None and user_profile.occupation is not None

    def user_profile_prompt(self, state: AgentState) -> AgentState:
        print("in user_profile_prompt")
        profile = state.get("user_profile")
        profile_json = json.dumps(profile.model_dump() if profile else {}, ensure_ascii=False)
        prompt = ChatPromptTemplate.from_template(ONBOARDING_USER_CREATE_PROMPT)


        rendered = prompt.format(user_profile=profile_json, user_input=state['messages'][-1].content)
        messages = [rendered] + state['messages']
        model_message = self.model.invoke(messages)
        return {'messages': [model_message]}

    def user_profile_extraction(self, state: AgentState) -> AgentState:
        print("in user_profile_extraction")
        parser = JsonOutputParser()
        messages = [SystemMessage(content=EXTRACTION_USER_CREATE_PROMPT)] + state["messages"]
        model_message = self.model.invoke(messages)
        content = model_message.content
        data = parser.parse(content)

        profile = UserProfile(
            name=data.get("name"),
            age=int(data.get("age")) if data.get("age") else None,
            gender=data.get("gender"),
            occupation=data.get("occupation"),
        )


        return {"messages": [model_message], "user_profile": profile} 

In [88]:
agent=Agent(checkpointer=memory)

In [91]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content="Hello. 30 years, male"),
    AIMessage(content="Please tell your occupation"),
    HumanMessage(content="I am a software engineer")
]

thread = {"configurable": {"thread_id": "1"}}
for event in agent.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

in user_profile_prompt
[AIMessage(content='Nice to meet you! Profile confirmed:\n- Age: 30\n- Gender: Male\n- Occupation: Software Engineer\n\nWhat would you like to do next?\n- Get tailored picks by genre, mood, or runtime (tell me your preference)\n- Check showtimes or streaming availability for a title (tell me the title, your country/region, and streaming service)\n- See today’s top picks based on your profile (brainy sci-fi, tech-thriller, etc.)\n\nIf you’d like quick starters, I can suggest a few:\n- The Martian\n- Ex Machina\n- Inception\n- Her\n- Blade Runner 2049\n- The Social Network\n\nTell me your vibe or choose an option above.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 672, 'prompt_tokens': 1887, 'total_tokens': 2559, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 153